# Homework - 09

In this homework, we'll deploy the dino or dragon model we trained in the 
[previous homework](https://github.com/alexeygrigorev/mlbookcamp-code/blob/master/course-zoomcamp/cohorts/2022/08-deep-learning/homework.md).

Download the model from here: 

https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5

In [1]:
from pathlib import Path
import wget

url = 'https://github.com/SVizor42/ML_Zoomcamp/releases/download/dino-dragon-model/dino_dragon_10_0.899.h5'
filename = Path(url.rsplit('/')[-1])

if not filename.exists():
    filename = Path(wget.download(url))

filename_tflite = Path(f'{filename.stem}.tflite')

## Question 1

Now convert this model from Keras to TF-Lite format.

In [2]:
import tensorflow as tf
from tensorflow import keras

model = keras.models.load_model(filename)
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
filename_tflite.write_bytes(tflite_model)
filename_tflite.stat().st_size

2022-11-21 11:10:33.227987: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-21 11:10:33.360284: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-21 11:10:33.887315: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/clamytoe/miniconda3/envs/py39/lib/
2022-11-21 11:10:33.887405: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_pl

INFO:tensorflow:Assets written to: /tmp/tmp2xv89mti/assets


INFO:tensorflow:Assets written to: /tmp/tmp2xv89mti/assets
2022-11-21 11:10:37.355157: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-11-21 11:10:37.355205: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-11-21 11:10:37.355722: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp2xv89mti
2022-11-21 11:10:37.356598: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2022-11-21 11:10:37.356614: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: /tmp/tmp2xv89mti
2022-11-21 11:10:37.360318: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-11-21 11:10:37.361019: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2022-11-21 11:10:37.413748: I tensorflow/cc/saved_model/loader.cc:213] Running initializatio

44866200

What's the size of the **converted** model?

* 21 Mb
* **43 Mb**
* 80 Mb
* 164 Mb

## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output.

In [3]:
interpreter = tf.lite.Interpreter(model_path=filename_tflite.name)
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']
target_size = tuple(interpreter.get_input_details()[0]['shape'][1:3])
print(f'{input_index=}')
print(f'{output_index=}')
print(f'{target_size=}')

input_index=0
output_index=13
target_size=(150, 150)


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


What's the output index for this model?

* 3
* 7
* **13**
* 24

## Preparing the image

You'll need some code for downloading and resizing images. You can use 
this code:

In [4]:
from io import BytesIO
from urllib import request

from PIL import Image


def download_image(url):
    with request.urlopen(url) as resp:
        buffer = resp.read()
    stream = BytesIO(buffer)
    img = Image.open(stream)
    return img


def prepare_image(img, target_size):
    if img.mode != 'RGB':
        img = img.convert('RGB')
    img = img.resize(target_size, Image.Resampling.NEAREST)
    return img

For that, you'll need to have `pillow` installed:

```bash
pip install pillow
```

Let's download and resize this image: 

https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg

Based on the previous homework, what should be the target size for the image?

In [5]:
img_url = 'https://upload.wikimedia.org/wikipedia/commons/thumb/d/df/Smaug_par_David_Demaret.jpg/1280px-Smaug_par_David_Demaret.jpg'
img_orig = download_image(img_url)
img = prepare_image(img_orig, target_size)

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

In [6]:
import numpy as np

x = np.array(img)
X = np.array([x / 255], dtype=np.float32)
p1r = X[0][0][0][0]
print(f'{p1r=}')

p1r=0.5529412


After the pre-processing, what's the value in the first pixel, the R channel?

* 0.3353411
* **0.5529412**
* 0.7458824
* 0.9654902

## Question 4

Now let's apply this model to this image.

In [7]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()

interpreter.get_tensor(output_index)[0][0]

0.82448614

What's the output of the model?

* 0.17049132
* 0.39009996
* 0.60146114
* **0.82448614**

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.

## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.9
COPY dino-vs-dragon-v2.tflite .
```

And pushed it to [`svizor42/zoomcamp-dino-dragon-lambda:v2`](https://hub.docker.com/r/svizor42/zoomcamp-dino-dragon-lambda/tags).


> Note: The image already contains a model and it's not the same model
> as the one we used for questions 1-4.

## Question 5

Download the base image `svizor42/zoomcamp-dino-dragon-lambda:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

In [8]:
!docker images svizor42/zoomcamp-dino-dragon-lambda

REPOSITORY                             TAG       IMAGE ID       CREATED      SIZE
svizor42/zoomcamp-dino-dragon-lambda   v2        20ef58b21a05   3 days ago   639MB


So what's the size of this base image?

* 139 Mb
* 329 Mb
* **639 Mb**
* 929 Mb

You can get this information when running `docker images` - it'll be in the "SIZE" column.

## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `dino-vs-dragon-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score this image: https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg

In [9]:
import requests

container_url = 'http://localhost:8080/2015-03-31/functions/function/invocations'
data = {
    "url": "https://upload.wikimedia.org/wikipedia/en/e/e9/GodzillaEncounterModel.jpg"
}
result = requests.post(container_url, json=data).json()
print(result)

{'prediction': 0.31950676441192627}


What's the output from the model?

* 0.12
* **0.32**
* 0.52
* 0.72

## Publishing it to AWS

Now you can deploy your model to AWS!

* Publish your image to ECR
* Create a lambda function in AWS, use the ECR image
* Give it more RAM and increase the timeout 
* Test it
* Expose the lambda function using API Gateway

This is optional and not graded.

## Publishing to Docker hub

This is just for reference, this is how we published our image to Docker hub:

```bash
docker build -t zoomcamp-dino-dragon-lambda .
docker tag zoomcamp-dino-dragon-lambda:latest svizor42/zoomcamp-dino-dragon-lambda:v2
docker push svizor42/zoomcamp-dino-dragon-lambda:v2
```

## Submit the results

* Submit your results here: https://forms.gle/Pnx563ELg9jgjxHX6
* You can submit your solution multiple times. In this case, only the last submission will be used 
* If your answer doesn't match options exactly, select the closest one


## Deadline

The deadline for submitting is **28 November 2022 (Monday), 23:00 CEST (Berlin time)**. 

After that, the form will be closed.